In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import keras
from tensorflow import keras

In [8]:
# GPU 장치 목록 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow는 GPU를 사용할 수 있습니다.")
    for gpu in gpus:
        print(f"사용 가능한 GPU: {gpu}")

    # gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    # print(gpus)
    # if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)
else:
    print("TensorFlow는 GPU를 사용할 수 없습니다.")

TensorFlow는 GPU를 사용할 수 있습니다.
사용 가능한 GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-08-14 01:30:46.520805: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 01:30:46.635984: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 01:30:46.636032: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.


In [9]:
# MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [10]:
# 데이터 전처리
x_train = x_train.reshape(-1, 32, 32, 3).astype('float32') / 255.0
x_test = x_test.reshape(-1, 32, 32, 3).astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

In [11]:
def seperate_convolution(X, f, nx, ny, s1, s2):
    X = keras.layers.Conv2D(filters=nx, kernel_size=(f, f), strides=(s1, s1), padding='same')(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.Conv2D(filters=ny, kernel_size=(1, 1), strides=(s2, s2), padding='same')(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    return X

In [41]:
# inputs = tf.keras.Input(shape=(32, 32, 3)) #imagenet이 아니라 cifar-100이기 때문에 로 값을 더 줄인다고 생각 7x7에 도달하면 그 떄 avg pool 진행

# first_conv = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')(inputs)
# first_conv = keras.layers.BatchNormalization()(first_conv)
# first_conv = keras.layers.Activation('relu')(first_conv)
# sep_conv_16_1 = seperate_convolution(first_conv, 3, 32, 64, 1, 1)
# sep_conv_8_1 = seperate_convolution(sep_conv_16_1, 3, 64, 128, 2, 1)
# sep_conv_8_2 = seperate_convolution(sep_conv_8_1, 3, 128, 128, 1, 1)
# sep_conv_4_1 = seperate_convolution(sep_conv_8_2, 3, 128, 256, 2, 1)
# sep_conv_4_2 = seperate_convolution(sep_conv_4_1, 3, 256, 256, 1, 1)
# sep_conv_2 = seperate_convolution(sep_conv_4_2, 3, 256, 512, 2, 1)
# for i in range(5):
#     sep_conv_2 = seperate_convolution(sep_conv_2, 3, 512, 512, 1, 1)
# sep_conv_1_1 = seperate_convolution(sep_conv_2, 3, 512, 1024, 2, 1)
# sep_conv_1_2 = seperate_convolution(sep_conv_1_1, 3, 1024, 1024, 1, 1)
# avg_pool = keras.layers.AveragePooling2D((1, 1), strides=(1, 1), padding='same')(sep_conv_1_2)
# avg_pool = keras.layers.Flatten()(avg_pool)
# fc = keras.layers.Dense(100, 'softmax')(avg_pool)

inputs = tf.keras.Input(shape=(32, 32, 3)) #imagenet이 아니라 cifar-100이기 때문에 로 값을 더 줄인다고 생각 7x7에 도달하면 그 떄 avg pool 진행

first_conv = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=2, padding='same')(inputs)
first_conv = keras.layers.BatchNormalization()(first_conv)
first_conv = keras.layers.Activation('relu')(first_conv)
sep_conv_16_1 = seperate_convolution(first_conv, 3, 32, 64, 1, 1)
sep_conv_8_1 = seperate_convolution(sep_conv_16_1, 3, 64, 128, 1, 1)
sep_conv_8_2 = seperate_convolution(sep_conv_8_1, 3, 128, 128, 1, 1)
sep_conv_4_1 = seperate_convolution(sep_conv_8_2, 3, 128, 256, 1, 1)
sep_conv_4_2 = seperate_convolution(sep_conv_4_1, 3, 256, 256, 1, 1)
sep_conv_2 = seperate_convolution(sep_conv_4_2, 3, 256, 512, 2, 1)
for i in range(5):
    sep_conv_2 = seperate_convolution(sep_conv_2, 3, 512, 512, 1, 1)
sep_conv_1_1 = seperate_convolution(sep_conv_2, 3, 512, 1024, 1, 1)
sep_conv_1_2 = seperate_convolution(sep_conv_1_1, 3, 1024, 1024, 1, 1)
avg_pool = keras.layers.AveragePooling2D((8, 8), strides=(1, 1), padding='valid')(sep_conv_1_2)
avg_pool = keras.layers.Flatten()(avg_pool)
fc = keras.layers.Dense(100, 'softmax')(avg_pool)



In [42]:
# 모델 생성
model = tf.keras.Model(inputs=inputs, outputs=fc)

# 모델 컴파일
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_237 (Conv2D)             │ (None, 16, 16, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_237         │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_237 (Activation)     │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_238 (Conv2D)             │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_238         │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_238 (Activation)     │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_239 (Conv2D)             │ (None, 16, 16, 64)     │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_239         │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_239 (Activation)     │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_240 (Conv2D)             │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_240         │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_240 (Activation)     │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_241 (Conv2D)             │ (None, 16, 16, 128)    │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_241         │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_241 (Activation)     │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_242 (Conv2D)             │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_242         │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_242 (Activation)     │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_243 (Conv2D)             │ (None, 16, 16, 128)    │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_243         │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 28,411,268 (108.38 MB)

 Trainable params: 28,389,380 (108.30 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [44]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.3680 - loss: 2.3937 - val_accuracy: 0.2636 - val_loss: 3.0827
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.3915 - loss: 2.2780 - val_accuracy: 0.3010 - val_loss: 2.8526
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 70ms/step - accuracy: 0.4131 - loss: 2.1617 - val_accuracy: 0.2822 - val_loss: 3.0940
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.4415 - loss: 2.0543 - val_accuracy: 0.2959 - val_loss: 3.0080
Epoch 5/10
180/625 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - accuracy: 0.4710 - loss: 1.9211

In [25]:
# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3759 - loss: 4.6721
Test Loss: 4.7139
Test Accuracy: 0.3750
